<a href="https://colab.research.google.com/github/AhnafZahin/Computer-Networks/blob/main/10_Users_Modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import scipy.io as sio
import numpy as np

import tensorflow.compat.v1 as tf
import time

In [ ]:
tf.compat.v1.disable_eager_execution()
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
N = 10
num_train = 100000
num_test = 10000
epochs = 100
batch_size = 256
learning_rate = 0.0001

In [ ]:
load = sio.loadmat('data/Train_data_%d_%d.mat' % (N, num_train))
loadTest = sio.loadmat('data/Test_data_%d_%d.mat' % (N, num_test))

In [ ]:
Htrain = load['Xtrain']
Ptrain = load['Ytrain']
H_test = loadTest['X']
P_test = loadTest['Y']
timeW = loadTest['swmmsetime']
swmmsetime = timeW[0, 0]

In [ ]:
weights = {
    "w_1": tf.Variable(tf.random_normal([N * N, 100], stddev=0.1)),
    "w_2": tf.Variable(tf.random_normal([100, 100], stddev=0.1)),
    "w_3": tf.Variable(tf.random_normal([100, 100], stddev=0.1)),
    "w_4": tf.Variable(tf.random_normal([100, 100], stddev=0.1)),
    "w_out": tf.Variable(tf.random_normal([100, N])),
}

In [ ]:
biases = {
    "b_1": tf.Variable(tf.random_normal([100], stddev=0.1)),
    "b_2": tf.Variable(tf.random_normal([100], stddev=0.1)),
    "b_3": tf.Variable(tf.random_normal([100], stddev=0.1)),
    "b_4": tf.Variable(tf.random_normal([100], stddev=0.1)),
    "b_out": tf.Variable(tf.random_normal([N])),
}

In [ ]:
def network(input_data):
    x = tf.nn.dropout(x, tf.placeholder(tf.float32))

    conv1 = tf.nn.conv2d(input, weights['1_1'], strides=[1, 1, 1, 1], padding='SAME')
    conv1 = tf.nn.bias_add(conv1, biases['b_1'])
    relu1 = tf.nn.relu(conv1)

    h_layer_1 = tf.add(
        tf.matmul(input_data, weights["w_1"]), biases["b_1"]
    )  # h1 = wx+b
    h_layer_1 = tf.nn.relu(h_layer_1)

    conv2 = tf.nn.conv2d(relu1, weights['w_2'], strides=[1, 1, 1, 1], padding='SAME')
    conv2 = tf.nn.bias_add(conv2, biases['b_2'])
    relu2 = tf.nn.relu(conv2)

    h_layer_2 = tf.add(tf.matmul(h_layer_1, weights["w_2"]), biases["b_1"])
    h_layer_2 = tf.nn.relu(h_layer_2)

    conv3 = tf.nn.conv2d(relu2, weights['w_3'], strides=[1, 1, 1, 1], padding='SAME')
    conv3 = tf.nn.bias_add(conv3, biases['b_3'])
    relu3 = tf.nn.relu(conv3)

    h_layer_3 = tf.add(tf.matmul(h_layer_2, weights["w_3"]), biases["b_3"])
    h_layer_3 = tf.nn.relu(h_layer_3)

    conv4 = tf.nn.conv2d(relu3, weights['w_4'], strides=[1, 1, 1, 1], padding='SAME')
    conv4 = tf.nn.bias_add(conv4, biases['b_4'])
    relu4 = tf.nn.relu(conv4)

    output = tf.matmul(h_layer_3, weights["w_out"]) + biases["b_out"]
    output = tf.nn.relu6(output) / 6

    return output

In [ ]:
valid_split = 0.1

total_sample_size = num_train
validation_sample_size = int(total_sample_size * valid_split)
training_sample_size = total_sample_size - validation_sample_size

In [ ]:
Htrain = np.reshape(Htrain, (total_sample_size, N * N))
number_input = N * N
number_output = N

H_train = Htrain[0:training_sample_size, :]
P_train = Ptrain[0:training_sample_size, :]
H_val = Htrain[training_sample_size:total_sample_size, :]
P_val = Ptrain[training_sample_size:total_sample_size, :]

In [ ]:
x = tf.placeholder(tf.float32, [None, N * N])
y = tf.placeholder(tf.float32, [None, N])

In [ ]:
total_batch = int(total_sample_size / batch_size)

In [ ]:
x_bar = network(x)
loss = tf.reduce_mean(tf.square(x_bar - y))

optimizer = tf.train.AdamOptimizer(learning_rate)
objective = optimizer.minimize(loss)

In [ ]:
save_data = np.zeros((epochs, 3))

session = tf.InteractiveSession()
tf.global_variables_initializer().run()

2022-11-20 19:52:24.639442: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


In [ ]:
start_time = time.time()
for e in range(epochs):
    for b in range(total_batch):
        batch = np.random.randint(training_sample_size, size=batch_size)
        _, training_cost = session.run(
            [objective, loss], feed_dict={x: H_train[batch, :], y: P_train[batch, :]}
        )
        save_data[e, 0] = training_cost
    validation_cost = session.run(loss, feed_dict={x: H_val, y: P_val})
    save_data[e, 1] = validation_cost
    save_data[e, 2] = e
    if e % 2 == 0:
        print(
            "\n %d " % e,
            " %f " % (training_cost),
            " %f " % (validation_cost),
            " %f " % (time.time() - start_time),
        )
    else:
        print("#", end="")
print("training time: %0.2f s" % (time.time() - start_time))
sio.savemat(
    "MSETime_%d_%d_%d" % (N, batch_size, learning_rate * 10000),
    {"train": save_data[:, 0], "validation": save_data[:, 1], "epoch": save_data[:, 2]},
)


 0   0.166241   0.167453   0.553056 
#
 2   0.077305   0.078712   1.800498 
#
 4   0.065804   0.066919   3.238195 
#
 6   0.066149   0.065561   4.619155 
#
 8   0.070018   0.064752   6.170786 
#
 10   0.063415   0.064509   7.652471 
#
 12   0.057866   0.063988   8.976901 
#
 14   0.061716   0.064735   10.274272 
#
 16   0.063122   0.063957   11.566386 
#
 18   0.061003   0.064309   12.867814 
#
 20   0.064910   0.063915   14.176314 
#
 22   0.069103   0.064248   15.478377 
#
 24   0.067858   0.063933   16.799587 
#
 26   0.065324   0.063513   18.206083 
#
 28   0.063122   0.063287   19.599008 
#
 30   0.061661   0.063480   20.941553 
#
 32   0.062936   0.063255   22.319263 
#
 34   0.059549   0.063086   23.738479 
#
 36   0.060656   0.062790   25.086898 
#
 38   0.055588   0.062623   26.405794 
#
 40   0.059540   0.062824   27.720677 
#
 42   0.060565   0.062893   29.075211 
#
 44   0.061195   0.062057   30.417854 
#
 46   0.063739   0.062096   31.795070 
#
 48   0.056876   0.061559  

In [ ]:
start_time = time.time()
H_test_r = np.reshape(H_test, (num_test, N * N))
predicted_power = session.run(x_bar, feed_dict={x: H_test_r, y: P_test})
pred_time = time.time() - start_time
print(predicted_power.shape)
predicted_power = np.reshape(predicted_power, (num_test, N))


(10000, 10)


In [ ]:
def IC_sum_rate(H, p, var_noise):
    H = np.square(H)
    fr = np.diag(H) * p
    ag = np.dot(H, p) + var_noise - fr
    y = np.sum(np.log(1 + fr / ag))
    return y

In [ ]:
def np_sum_rate(X, Y):
    avg = 0
    n = X.shape[0]
    for i in range(n):
        avg += IC_sum_rate(X[i, :, :], Y[i, :], 1) / n
    return avg

In [ ]:
sum_rate_reg = np_sum_rate(H_test, predicted_power) * np.log2(np.exp(1))
sum_rate_swmmse = np_sum_rate(H_test, P_test) * np.log2(np.exp(1))

In [ ]:
print("sum rate for regularized cnn", sum_rate_reg)
print("sum rate for SWMMSE", sum_rate_swmmse)
print("accuracy %f%%" % (sum_rate_reg / sum_rate_swmmse * 100))
print(swmmsetime)

sum rate for regularized cnn 2.404347158001954
sum rate for SWMMSE 2.675864821168107
accuracy 82.853087%
5.9626994132995605
